In [4]:
import getpass
import heapq
import time
from binance.client import Client
api_key = getpass.getpass('api_key:')
api_secret = getpass.getpass('api_secret:')

api_key:········
api_secret:········


In [174]:
""" Define Functions """

def calc_gain(t,price_dict):
    
    """ Calculate price difference version 1/19/2018"""
    
    # Compute prices for triangle
    price1 = float(price_dict[t[0]])
    if t[0][0:len(root)] != root:
        price1 = 1/price1

    price2 = float(price_dict[t[1]])
    m1 = t[0].replace(root,"")
    if t[1][0:len(m1)] != m1:
        price2 = 1/price2

    price3 = float(price_dict[t[2]])
    m2 = t[2].replace(root,"")
    if t[2][0:len(m2)] != m2:
        price3 = 1/price3

    price_final = (price1 * price2 * price3)
    return price_final

def price_api():
    
    """ API Request """
    
    client = Client(api_key, api_secret)
    
    # Get all symbol prices
    prices = client.get_all_tickers()
    
    # Organize data
    symbols = []
    price_dict = {}
    for p in prices:
        symbols.append(p['symbol'])
        price_dict.update({p['symbol']:p['price']})
    return price_dict, symbols
    
def get_trades(root, price_dict, symbols):
    

    """ Get trades version 1/19/2018 """
    
    # Explore medium 1
    md1 = [ k for k in symbols if k.find(root) != -1]

    # Explore medium 2
    md2p = {}
    for m in md1:
        n = m.replace(root,"")
        md2t = [ k for k in symbols if k.find(n) != -1 and k.find(root) == -1]
        md2p.update({n:md2t})

    # Validate triangle
    md2 = {}
    for m in md2p:
        n = md2p[m]
        n2 = []
        for item in n:
            n2.append(item.replace(m,""))
        for item in n2:
            vset = [ k for k in symbols if k.find(item) != -1 and k.find(root) != -1]
            if len(vset) == 1:
                md2.update({m:vset})

    # Format triangle
    triangle = []
    for m in md2:
        first_trade = [ k for k in symbols if k.find(m) != -1 and k.find(root) != -1][0]
        last_trade = md2[m][0]
        medium2 = last_trade.replace(root,"")
        middle_trade = [ k for k in symbols if k.find(m) != -1 and k.find(medium2) != -1][0]

        """This excludes VIB and VIBE currencies. Implemented for VIBETH and VIBE confusion."""
        if first_trade.find("VIB") == -1 and middle_trade.find("VIB") == -1:
            triangle.append([first_trade,middle_trade,last_trade])

    # Compute gains
    gains = {}
    for t in triangle:
        
        price_final = abs(1-(calc_gain(t,price_dict)))
        gains.update({str(t):price_final})

    # Find top n trades
    topn = heapq.nlargest(10, gains, key=gains.get)
    
    trades = {}
    counter = 1
    # Find trade direction
    for t in topn:
    
        t = t.replace("[","")
        t = t.replace("]","")
        t = t.replace("'","")
        t = t.replace(" ","")
        t = t.split(",")
        price_final = calc_gain(t,price_dict)
        #print(t)
        #print(price_final)

        if price_final < 1:
            temp = []
            temp.append(t[2])
            temp.append(t[1])
            temp.append(t[0])
            t = temp
            price_final = calc_gain(t,price_dict)
            #print("Flip")
        price_abs = abs(price_final-1)
        #print(t)
        #print(price_abs)
        trades.update({counter:{"name":t,"gain":price_abs}})
        counter = counter + 1
    return trades



In [199]:
""" Define Functions """

def calc_depth(root, t, elimit):

    price_dict2 = {}
    price_dict3 = {}

    # Compute prices for triangle
    price1 = float(price_dict[t[0]])
    if t[0][0:len(root)] != root:
        price1 = 1/price1

    price2 = float(price_dict[t[1]])
    m1 = t[0].replace(root,"")
    if t[1][0:len(m1)] != m1:
        price2 = 1/price2

    price3 = float(price_dict[t[2]])
    m2 = t[2].replace(root,"")
    if t[2][0:len(m2)] != m2:
        price3 = 1/price3

    elimit1 = elimit
    elimit2 = elimit1 * price1
    elimit3 = elimit2 * price2
    elimit = [elimit1, elimit2, elimit3]

    for i in range(3):

        esum = 0.0
        ecounter = 0

        """ API Call """
        depth = client.get_order_book(symbol=t[i])
        time.sleep(.15)
        namevalue = client.get_symbol_ticker(symbol=t[i])
        time.sleep(.15)

        while esum < elimit[i]:
            eitem = depth['asks'][ecounter]
            price = float(eitem[0])
            esum = esum + price * float(eitem[1])
            ecounter = ecounter + 1
            
        #print(esum)
        #print("final price")
        #print(price)
        #print("indicator price")
        #print(namevalue['price'])
        price_dict2.update({t[i]:price})
        price_dict3.update({t[i]:{'price':price, 'quant':elimit[i]}})   
    new_gain = calc_gain(t,price_dict2)
    return new_gain, price_dict3

In [200]:
""" Define Functions """

def get_balance(root):
    
    """ API Call """
    balance = client.get_asset_balance(asset=root)
    time.sleep(.15)
    
    return balance

def order(root, price_dict2, t, quant):
    
    for i in range(3):
        
        order = client.order_limit_buy(symbol=t[i], quantity=100, price='0.00001')


In [202]:
# Call functions
'''
price_api() - get updated price list
get_trades(root, price_dict, symbols) - find triangular trade opportunities
get_depth(root, t, elimit) - finds market depth adjusted opportunity 
get_balance(root) - return root balance
order(root, price_dict2, t, quant) - live triangle trading function

'''
start = time.time()

root = 'ETH'

balance = {}

price_dict, symbols = price_api()
trades = get_trades(root, price_dict, symbols)

for i in range(1,3):
    iname = trades[i]['name']
    print(iname)
    print(trades[i]['gain'])
    
    adj_gain, price_dict3 = calc_depth(root, iname, .05)
    print("Adjusted gain:")
    print(adj_gain)

end = time.time()
print("Machine time: " + str(end - start))


['RLCETH', 'RLCBNB', 'BNBETH']
0.024885068952664957
Adjusted gain:
1.0191429730726669
['NAVETH', 'NAVBNB', 'BNBETH']
0.02007509887553316
Adjusted gain:
1.0141494286809816
Machine time: 5.183563470840454


In [184]:
print(price_dict2)

{'RCNBNB': 0.02689, 'RCNETH': 0.00032773, 'BNBETH': 0.012511}


In [161]:
def calc_depth(root,t):

    price_dict2 = {}

    # Compute prices for triangle
    price1 = float(price_dict[t[0]])
    if t[0][0:len(root)] != root:
        price1 = 1/price1

    price2 = float(price_dict[t[1]])
    m1 = t[0].replace(root,"")
    if t[1][0:len(m1)] != m1:
        price2 = 1/price2

    price3 = float(price_dict[t[2]])
    m2 = t[2].replace(root,"")
    if t[2][0:len(m2)] != m2:
        price3 = 1/price3

    elimit1 = .1
    elimit2 = elimit1 * price1
    elimit3 = elimit2 * price2
    elimit = [elimit1, elimit2, elimit3]

    for i in range(3):

        esum = 0.0
        ecounter = 0

        """ API Call """
        depth = client.get_order_book(symbol=t[i])
        time.sleep(.2)
        namevalue = client.get_symbol_ticker(symbol=t[i])
        time.sleep(.2)

        while esum < elimit[i]:
            eitem = depth['asks'][ecounter]
            price = float(eitem[0])
            esum = esum + price * float(eitem[1])
            ecounter = ecounter + 1

        print(esum)
        print("final price")
        print(price)
        print("indicator price")
        print(namevalue['price'])
        price_dict2.update({t[i]:price})
        new_gain = calc_gain(t,price_dict2)
        return new_gain

0.9844987599999999
final price
0.00032773
indicator price
0.00032719
370.2901613
final price
0.02689
indicator price
0.02610000
12.782864029999999
final price
0.012511
indicator price
0.01251100


In [162]:
print(t)
print(calc_gain(t,price_dict))
print(calc_gain(t,price_dict2))

['RCNETH', 'RCNBNB', 'BNBETH']
0.9980045233656286
1.0265181399322614
